In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

In [10]:
import os
import nltk
from nltk.tokenize import sent_tokenize

# Load the text file
file_path = "resources/9thcomputerocr.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.readlines()

# Optional: Install NLTK if not already installed
# !pip install nltk
# nltk.download("punkt")

# Split the text into chunks (e.g., by sentences)
sentences = sent_tokenize(text)
chunks = [" ".join(sentences[i:i+5]) for i in range(0, len(sentences), 5)]  # Customize chunk size as needed


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\abdul/nltk_data'
    - 'c:\\Users\\abdul\\miniconda3\\envs\\fyp\\nltk_data'
    - 'c:\\Users\\abdul\\miniconda3\\envs\\fyp\\share\\nltk_data'
    - 'c:\\Users\\abdul\\miniconda3\\envs\\fyp\\lib\\nltk_data'
    - 'C:\\Users\\abdul\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Create a TfidfVectorizer to encode the chunks
vectorizer = TfidfVectorizer()
chunk_vectors = vectorizer.fit_transform(chunks)

# Function to find the most relevant chunk given a query
def retrieve_relevant_chunk(query):
    query_vector = vectorizer.transform([query])
    similarities = np.dot(chunk_vectors, query_vector.T).toarray()
    # Find the chunk with the highest similarity score
    most_relevant_index = np.argmax(similarities)
    return chunks[most_relevant_index]


In [ ]:
from groq import Groq

# Load environment variable for API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def ask_question(question):
    # Retrieve the relevant chunk
    relevant_chunk = retrieve_relevant_chunk(question)
    
    # Formulate a prompt using the retrieved context
    messages = [
        {"role": "system", "content": "You are an assistant knowledgeable in computer science topics."},
        {"role": "user", "content": f"Context: {relevant_chunk}"},
        {"role": "user", "content": f"Question: {question}"}
    ]
    
    # Query the model
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.1-70b-versatile"
    )
    
    # Return the generated answer
    return chat_completion.choices[0].message.content

# Example question
response = ask_question("What is the purpose of an operating system?")
print(f"Answer: {response}")
